# Load
<hr>

## Use SQLAlchemy to build a relational database

The purpose of this notebook is to create table anchor points (Classes) in python using SQLAlchemy by referencing the table structures from the saved databases in the "final_datasets" folder. The goal is to load the data from the csv files into our database.

In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 

In [2]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [3]:
# Creates Classes which will serve as the anchor points for our Tables
class Books(Base):
    __tablename__ = 'books_info'
    isbn = Column(String(100), primary_key=True)
    book_name = Column(String(500))
    authors = Column(String(255))
    rating = Column(Float)


class ISBN13(Base):
    __tablename__ = 'isbn13'
    isbn = Column(String(100))
    isbn13 = Column(String(100), primary_key=True)
    title = Column(String(500))

### Create and load instances of data for both Books and ISBN13 classes within for loops:

In [4]:
# Create Database Connection
database_path = '../database/goodreads.sqlite'
engine = create_engine(f'sqlite:///{database_path}')
conn = engine.connect()

In [5]:
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db (if applicable)
# Base.metadata.drop_all(engine)

In [6]:
# Check for tables
engine.table_names()

['books_info', 'isbn13']

#### Books

In [7]:
books_info_df = pd.read_csv('../final_datasets/books_info.csv')
books_info_df.head(2)

,isbn,book_name,authors,rating
0,0439682584,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",J.K. Rowling,4.78
1,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.79


##### Use pandas to load csv converted books_info df into database

In [8]:
books_info_df.to_sql(name='books_info', con=engine, if_exists='append', index=False)

<br>

#### ISBN13

In [9]:
isbn13_df = pd.read_csv('../final_datasets/isbn13.csv')
isbn13_df.head(2)

,isbn,isbn13,title
0,0312853122,9780312853129,W.C. Fields: A Life on Film
1,0850308712,9780850308716,Runic Astrology: Starcraft and Timekeeping in ...


##### Use pandas to load csv converted isbn13 df into database

In [10]:
isbn13_df.to_sql(name='isbn13', con=engine, if_exists='append', index=False)

<hr>

### Confirm data has been added by querying the customer_location table

In [11]:
# "books_info" table
check_books_info = pd.read_sql_query('select * from books_info', con=engine)
check_books_info.head()

,isbn,book_name,authors,rating
0,0439682584,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",J.K. Rowling,4.78
1,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.79
2,0439827604,"Harry Potter Collection (Harry Potter, #1-6)",J.K. Rowling,4.73
3,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.37
4,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.37


In [12]:
# Check number of rows
check_books_info.shape

(757407, 4)

In [13]:
# "isbn13" table
check_isbn13 = pd.read_sql_query('select * from isbn13', con=engine)
check_isbn13.head()

,isbn,isbn13,title
0,0312853122,9780312853129,W.C. Fields: A Life on Film
1,0850308712,9780850308716,Runic Astrology: Starcraft and Timekeeping in ...
2,1599150603,9781599150604,The Aeneid for Boys and Girls
3,0425040887,9780425040881,The Wanting of Levine
4,1934876569,9781934876565,All's Fairy in Love and War (Avalon: Web of Ma...


In [14]:
# Check number of rows
check_isbn13.shape

(235058, 3)

<strong>Check how many overlapped ISBN-10 ("isbn") there are</strong>

In [29]:
# Check how many overlapped ISBN-10 ("isbn") there are
merged_df = pd.merge(books_info_df, isbn13_df, on='isbn', how='inner')
print(merged_df.shape)

(39651, 6)


### Perform two example queries for two hypothetical analytical scenarios

In [16]:
# We will create a Session Object to perform the queries
from sqlalchemy.orm import Session
session = Session(bind=engine)

#### Scenario 1
We are in a bookshop and we want to buy some awesome books! These two datasets are all we have in our tool box and we want to use them to find the books that have the top 10 highest ratings  and their ISBN-13 codes. Assume the bookshop Boss is a robot and he/she can only recognize ISBN-13 as inputs to find books we're looking for.

In [44]:
# Perform a query that involves joins and sorting
results1 = session.query(ISBN13.isbn13, Books.book_name, Books.rating).\
                        filter(Books.isbn == ISBN13.isbn).\
                        order_by(Books.rating.desc()).limit(10).all()

for result in results1:
    print(f'ISBN-13: {result[0]}; Book: {result[1]}; Rating: {result[2]}')

ISBN-13: 9781851967773; Book: The Works of Elizabeth Gaskell, Part I; Rating: 5.0
ISBN-13: 9780006385202; Book: Friends of Footprints: How "Footprints" Has Left Its Imprint on Readers Around the World; Rating: 5.0
ISBN-13: 9780691095158; Book: Where to Watch Birds in Central America, Mexico, and the Caribbean; Rating: 5.0
ISBN-13: 9780198162841; Book: Vaughan Williams's Ninth Symphony; Rating: 5.0
ISBN-13: 9780805854893; Book: Twelve Years of Correspondence with Paul Meehl: Tough Notes from a Gentle Genius; Rating: 5.0
ISBN-13: 9780160019999; Book: From the Volturno to the Winter Line, 6 October - 15 November 1943; Rating: 5.0
ISBN-13: 9780856675256; Book: Vivienne Westwood: A London Fashion; Rating: 5.0
ISBN-13: 9781565070899; Book: Proactive Parenting: The Only Approach That Works; Rating: 5.0
ISBN-13: 9780595411030; Book: Shifting Sands: A Clash of Cultures; Rating: 5.0
ISBN-13: 9780028723105; Book: Black Popular Music In America: The singers, songwriters and musicians who pioneered

#### Scenario 2
We are curious to see the top 10 authors with the highest <strong>average</strong> books rating that has written more than one book and the number of books they have written (at least in our database).

In [45]:
# Perform a query that involves aggregating and sorting
results2 = session.query(Books.authors,\
                         func.avg(Books.rating),\
                         func.count(Books.book_name)).\
                        group_by(Books.authors).\
                        having(func.count(Books.book_name) > 10).\
                        order_by(func.avg(Books.rating).desc()).limit(10).all()

for result in results2:
    print(f'Author/Artist: {result[0]}; Average Rating: {round(result[1], 2)}; Number of Works:{result[2]}')

Author/Artist: Clive   Barker; Average Rating: 4.89; Number of Works:11
Author/Artist: Guns N' Roses; Average Rating: 4.77; Number of Works:12
Author/Artist: Metallica; Average Rating: 4.7; Number of Works:13
Author/Artist: Bill Watterson; Average Rating: 4.69; Number of Works:26
Author/Artist: Adi Da Samraj; Average Rating: 4.69; Number of Works:18
Author/Artist: Bentley Boyd; Average Rating: 4.68; Number of Works:11
Author/Artist: Joel S. Goldsmith; Average Rating: 4.66; Number of Works:21
Author/Artist: Frédéric Chopin; Average Rating: 4.64; Number of Works:15
Author/Artist: Salaheddin Ali Nader Shah Angha; Average Rating: 4.61; Number of Works:14
Author/Artist: Robert N. Pripps; Average Rating: 4.61; Number of Works:12


<br>
Comment: Unfortunately it looks like the books_info (Kaggle) dataset originally created from "goodreads.com" is not only limited to <em>writers</em> or "<em>book authors</em>" if you will. The returned results that meet our criteria included <strong>Guns N' Roses</strong> and  <strong>Metallica</strong> who are bands... There are also cartoonist, composer, and even <em>Spirital Teacher</em>. Only a few of them are actually recognized as either writer, author, or playwright.

In [46]:
# Close the session
session.close() 